<a href="https://colab.research.google.com/github/qianyu-berkeley/NLP_study/blob/main/nlp_tasks/Fast_tokenizers_in_the_QA_pipeline_(PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fast tokenizers in the QA pipeline (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00


# Question and Answering use case using pipeline API

* pipeline: `pipeline("question-answering")`
* Work with both short and long context

In [2]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch, and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "Which deep learning libraries back 🤗 Transformers?"
question_answerer(question=question, context=context)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.9802603125572205,
 'start': 78,
 'end': 106,
 'answer': 'Jax, PyTorch, and TensorFlow'}

In [3]:
long_context = """
🤗 Transformers: State of the Art NLP

🤗 Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction,
question answering, summarization, translation, text generation and more in over 100 languages.
Its aim is to make cutting-edge NLP easier to use for everyone.

🤗 Transformers provides APIs to quickly download and use those pretrained models on a given text, fine-tune them on your own datasets and
then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and
can be modified to enable quick research experiments.

Why should I use transformers?

1. Easy-to-use state-of-the-art models:
  - High performance on NLU and NLG tasks.
  - Low barrier to entry for educators and practitioners.
  - Few user-facing abstractions with just three classes to learn.
  - A unified API for using all our pretrained models.
  - Lower compute costs, smaller carbon footprint:

2. Researchers can share trained models instead of always retraining.
  - Practitioners can reduce compute time and production costs.
  - Dozens of architectures with over 10,000 pretrained models, some in more than 100 languages.

3. Choose the right framework for every part of a model's lifetime:
  - Train state-of-the-art models in 3 lines of code.
  - Move a single model between TF2.0/PyTorch frameworks at will.
  - Seamlessly pick the right framework for training, evaluation and production.

4. Easily customize a model or an example to your needs:
  - We provide examples for each architecture to reproduce the results published by its original authors.
  - Model internals are exposed as consistently as possible.
  - Model files can be used independently of the library for quick experiments.

🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question_answerer(question=question, context=long_context)

{'score': 0.9714871048927307,
 'start': 1892,
 'end': 1919,
 'answer': 'Jax, PyTorch and TensorFlow'}

# Question and Answering use case using AutoModel API

In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_checkpoint = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

inputs = tokenizer(question, context, return_tensors="pt")
outputs = model(**inputs)

In [5]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([1, 67]) torch.Size([1, 67])


In [6]:
print("----- start logits ------")
print(start_logits)
print("----- end logits ------")
print(end_logits)

----- start logits ------
tensor([[-4.4952, -6.4454, -4.7115, -7.0968, -7.0726, -7.4981, -5.5397, -4.1368,
         -5.9199, -5.4193, -1.5920, -1.0857, -5.0981, -2.9331, -3.4070,  2.2467,
          5.1563, -1.3602, -2.2209, -0.9686, -4.8112, -2.2527,  1.4383, 10.1211,
         -1.5311,  2.2685, -1.8951, -2.2108, -4.2142, -2.5571, -2.3252, -2.6046,
          1.7047, -1.9867, -1.7211, -0.5415, -2.0239, -4.4246, -5.1012, -4.4966,
         -7.8940, -6.7200, -4.6759, -6.3278, -4.8339, -5.1839, -3.3724, -7.4120,
         -8.1542, -4.4871, -7.4659, -4.3293, -4.2293, -3.1903, -7.9467, -5.2665,
         -7.5902, -5.0570, -7.4476, -7.9083, -6.5951, -7.4061, -8.8821, -7.6749,
         -6.9879, -7.0466, -5.4193]], grad_fn=<CloneBackward0>)
----- end logits ------
tensor([[-2.3958e+00, -7.0978e+00, -7.0745e+00, -6.3676e+00, -5.9532e+00,
         -7.9585e+00, -7.1869e+00, -3.6494e+00, -6.9677e+00, -5.1421e+00,
         -3.1757e+00, -1.1649e+00, -7.0748e+00, -5.2875e+00, -6.8611e+00,
         -5.1769

## Masking the context and `[SEP]` characters except `[CLS]` token

* Some QA models use `[CLS]` to indicate that the answer is not in the context.
* To apply a softmax afterward, we need to replace the logits we masked with a large negative number. e.g. use -10000, to get 0.0

In [7]:
import torch

sequence_ids = inputs.sequence_ids()
# Mask everything apart from the tokens of the context
mask = [i != 1 for i in sequence_ids]
# Unmask the [CLS] token
mask[0] = False
mask = torch.tensor(mask)[None]

start_logits[mask] = -10000
end_logits[mask] = -10000

In [8]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)[0]
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)[0]

In [9]:
print("------- start probability --------")
print(start_probabilities.shape)
print(start_probabilities)
print("------- end probability --------")
print(end_probabilities.shape)
print(end_probabilities)

------- start probability --------
torch.Size([67])
tensor([4.4531e-07, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.1185e-06, 1.3470e-05,
        2.4368e-07, 2.1236e-06, 1.3220e-06, 3.7722e-04, 6.9219e-03, 1.0237e-05,
        4.3289e-06, 1.5143e-05, 3.2463e-07, 4.1933e-06, 1.6808e-04, 9.9179e-01,
        8.6288e-06, 3.8557e-04, 5.9956e-06, 4.3725e-06, 5.8977e-07, 3.0929e-06,
        3.8999e-06, 2.9493e-06, 2.1940e-04, 5.4713e-06, 7.1354e-06, 2.3212e-05,
        5.2711e-06, 4.7788e-07, 2.4291e-07, 4.4467e-07, 1.4879e-08, 4.8133e-08,
        3.7169e-07, 7.1242e-08, 3.1735e-07, 2.2365e-07, 1.3685e-06, 2.4093e-08,
        1.1470e-08, 4.4891e-07, 2.2828e-08, 5.2562e-07, 5.8092e-07, 1.6419e-06,
        1.4114e-08, 2.0591e-07, 2.0161e-08, 2.5390e-07, 2.3251e-08, 1.4667e-08,
        5.4533e-08, 2.4235e-08, 5.5390e-09, 1.8524e-08, 3.6818e-08, 3.4721e-08,
        0.0000e+00], grad_fn=<SelectBackward0>)
------- end probabil

In [10]:
print(start_probabilities[:, None].shape)
print(end_probabilities[None, :].shape)

torch.Size([67, 1])
torch.Size([1, 67])


## Find the best start and end index

* To prevent start index larger than end index from argmax (meaning to avoid first word of the answer behind the last word in the text), We will compute the probabilities of each possible start_index and end_index where start_index <= end_index, then take the tuple (start_index, end_index) with the highest probability.
* compute all the products `start_probabilities[start_index]×end_probabilities[end_index]` where start_index <= end_index.
* Then we’ll mask the values where start_index > end_index by setting them to 0 (the other probabilities are all positive numbers). The `torch.triu()` function returns the upper triangular part of the 2D tensor passed as an argument, so it will do that masking for us because start_index <= end_index at the upper triangle

In [11]:
scores = start_probabilities[:, None] * end_probabilities[None, :]
print("---- Score ----")
print(scores.shape)
print(scores)

---- Score ----
torch.Size([67, 67])
tensor([[9.4340e-13, 0.0000e+00, 0.0000e+00,  ..., 1.1023e-12, 1.6345e-12,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [7.8000e-14, 0.0000e+00, 0.0000e+00,  ..., 9.1135e-14, 1.3514e-13,
         0.0000e+00],
        [7.3557e-14, 0.0000e+00, 0.0000e+00,  ..., 8.5944e-14, 1.2744e-13,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], grad_fn=<MulBackward0>)


In [12]:
scores = torch.triu(scores)
print("---- upper triangle only score")
print(scores.shape)
print(scores)

---- upper triangle only score
torch.Size([67, 67])
tensor([[9.4340e-13, 0.0000e+00, 0.0000e+00,  ..., 1.1023e-12, 1.6345e-12,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.1135e-14, 1.3514e-13,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 1.2744e-13,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], grad_fn=<TriuBackward0>)


In [13]:
max_index = scores.argmax().item()
print(f"max_index: {max_index}")
start_index = max_index // scores.shape[1]
end_index = max_index % scores.shape[1]
print(f"(start_index, end_index): ({start_index}, {end_index})")
print(scores[start_index, end_index])

max_index: 1576
(start_index, end_index): (23, 35)
tensor(0.9803, grad_fn=<SelectBackward0>)


In [14]:
inputs_with_offsets = tokenizer(question, context, return_offsets_mapping=True)
offsets = inputs_with_offsets["offset_mapping"]

start_char, _ = offsets[start_index]
_, end_char = offsets[end_index]
answer = context[start_char:end_char]

In [15]:
result = {
    "answer": answer,
    "start": start_char,
    "end": end_char,
    "score": scores[start_index, end_index],
}
result

{'answer': 'Jax, PyTorch, and TensorFlow',
 'start': 78,
 'end': 106,
 'score': tensor(0.9803, grad_fn=<SelectBackward0>)}

## Find top 5 answers

* Using the same process to find the top 5 start, end index
* Then get the tokens span between those indexs

In [16]:
values, indexes = scores.flatten().topk(k=5)
print(values)
print(indexes)

tensor([9.8026e-01, 8.2478e-03, 6.8415e-03, 1.3677e-03, 3.8109e-04],
       grad_fn=<TopkBackward0>)
tensor([1576, 1577, 1107, 1570, 1710])


In [17]:
top5_indexes = []

for i in range(len(indexes)):
    s = indexes[i].item() // scores.shape[1]
    e = indexes[i].item() % scores.shape[1]
    print(f" top {i+1} (start_index, end_index): ({s}, {e})")
    top5_indexes.append((s, e))
print(top5_indexes)

 top 1 (start_index, end_index): (23, 35)
 top 2 (start_index, end_index): (23, 36)
 top 3 (start_index, end_index): (16, 35)
 top 4 (start_index, end_index): (23, 29)
 top 5 (start_index, end_index): (25, 35)
[(23, 35), (23, 36), (16, 35), (23, 29), (25, 35)]


In [18]:
for s, e in top5_indexes:
    start_char, _ = offsets[s]
    _, end_char = offsets[e]
    answer = context[start_char:end_char]
    result = {
        "answer": answer,
        "start": start_char,
        "end": end_char,
        "score": scores[s, e],
    }
    print(result)

{'answer': 'Jax, PyTorch, and TensorFlow', 'start': 78, 'end': 106, 'score': tensor(0.9803, grad_fn=<SelectBackward0>)}
{'answer': 'Jax, PyTorch, and TensorFlow —', 'start': 78, 'end': 108, 'score': tensor(0.0082, grad_fn=<SelectBackward0>)}
{'answer': 'three most popular deep learning libraries — Jax, PyTorch, and TensorFlow', 'start': 33, 'end': 106, 'score': tensor(0.0068, grad_fn=<SelectBackward0>)}
{'answer': 'Jax, PyTorch', 'start': 78, 'end': 90, 'score': tensor(0.0014, grad_fn=<SelectBackward0>)}
{'answer': 'PyTorch, and TensorFlow', 'start': 83, 'end': 106, 'score': tensor(0.0004, grad_fn=<SelectBackward0>)}


## AutoModel API for Long Context

* Use `trunction="only_second"` strategy to truncate context
* To avoid loss of answer, use `return_overflowing_tokens=True` option in tokenizer

In [19]:
inputs = tokenizer(question, long_context)
print(len(inputs["input_ids"]))

461


In [20]:
inputs = tokenizer(question, long_context, max_length=384, truncation="only_second")
print(tokenizer.decode(inputs["input_ids"]))

[CLS] Which deep learning libraries back [UNK] Transformers? [SEP] [UNK] Transformers : State of the Art NLP [UNK] Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction, question answering, summarization, translation, text generation and more in over 100 languages. Its aim is to make cutting - edge NLP easier to use for everyone. [UNK] Transformers provides APIs to quickly download and use those pretrained models on a given text, fine - tune them on your own datasets and then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and can be modified to enable quick research experiments. Why should I use transformers? 1. Easy - to - use state - of - the - art models : - High performance on NLU and NLG tasks. - Low barrier to entry for educators and practitioners. - Few user - facing abstractions with just three classes to learn. - A unified A

In [21]:
sentence = "This sentence is not too long but we are going to split it anyway."
inputs = tokenizer(
    sentence, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

[CLS] This sentence is not [SEP]
[CLS] is not too long [SEP]
[CLS] too long but we [SEP]
[CLS] but we are going [SEP]
[CLS] are going to split [SEP]
[CLS] to split it anyway [SEP]
[CLS] it anyway. [SEP]


In [22]:
print(inputs.keys())

dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])


In [23]:
print(inputs["overflow_to_sample_mapping"])

[0, 0, 0, 0, 0, 0, 0]


In [24]:
sentences = [
    "This sentence is not too long but we are going to split it anyway.",
    "This sentence is shorter but will still get split.",
]
inputs = tokenizer(
    sentences, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)

print(inputs["overflow_to_sample_mapping"])

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


In [25]:
inputs = tokenizer(
    question,
    long_context,
    stride=128,
    max_length=384,
    padding="longest",
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

Those inputs will contain the input IDs and attention masks the model expects, as well as the offsets and the `overflow_to_sample_mapping`. Since those two are not parameters used by the model, we’ll pop them out of the inputs (and we won’t store the map, since it’s not useful here) before converting it to a tensor:

In [26]:
_ = inputs.pop("overflow_to_sample_mapping")
offsets = inputs.pop("offset_mapping")

inputs = inputs.convert_to_tensors("pt")
print(inputs["input_ids"].shape)

torch.Size([2, 384])


In [27]:
outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([2, 384]) torch.Size([2, 384])


**Notices the first dimension of the tensor is 2 now as we create chunks**

In [28]:
sequence_ids = inputs.sequence_ids()
# Mask everything apart from the tokens of the context
mask = [i != 1 for i in sequence_ids]
# Unmask the [CLS] token
mask[0] = False
# Mask all the [PAD] tokens
mask = torch.logical_or(torch.tensor(mask)[None], (inputs["attention_mask"] == 0))

start_logits[mask] = -10000
end_logits[mask] = -10000

In [29]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)

In [34]:
candidates = []
for start_probs, end_probs in zip(start_probabilities, end_probabilities):
    scores = start_probs[:, None] * end_probs[None, :]
    idx = torch.triu(scores).argmax().item()

    start_idx = idx // scores.shape[1]
    end_idx = idx % scores.shape[1]
    score = scores[start_idx, end_idx].item()
    candidates.append((start_idx, end_idx, score))

print(candidates)

[(0, 18, 0.3386707305908203), (173, 184, 0.9714868664741516)]


In [31]:
for candidate, offset in zip(candidates, offsets):
    start_token, end_token, score = candidate
    start_char, _ = offset[start_token]
    _, end_char = offset[end_token]
    answer = long_context[start_char:end_char]
    result = {"answer": answer, "start": start_char, "end": end_char, "score": score}
    print(result)

{'answer': '\n🤗 Transformers: State of the Art NLP', 'start': 0, 'end': 37, 'score': 0.3386707305908203}
{'answer': 'Jax, PyTorch and TensorFlow', 'start': 1892, 'end': 1919, 'score': 0.9714868664741516}


## Get top 5 most likely answer (not per chunk)

In [50]:
candidates = []
chunk = 0
for start_probs, end_probs in zip(start_probabilities, end_probabilities):
    scores = start_probs[:, None] * end_probs[None, :]
    scores = torch.triu(scores)
    values, indexes = scores.flatten().topk(k=5)
    top5_indexes = []
    for i in range(len(indexes)):
        s = indexes[i].item() // scores.shape[1]
        e = indexes[i].item() % scores.shape[1]
        print(f" top {i+1} (start_index, end_index): ({s}, {e})")
        score = scores[s, e].item()
        candidates.append((chunk, s, e, score))
    chunk += 1

candidates

 top 1 (start_index, end_index): (0, 18)
 top 2 (start_index, end_index): (0, 0)
 top 3 (start_index, end_index): (13, 18)
 top 4 (start_index, end_index): (17, 18)
 top 5 (start_index, end_index): (0, 19)
 top 1 (start_index, end_index): (173, 184)
 top 2 (start_index, end_index): (173, 185)
 top 3 (start_index, end_index): (166, 184)
 top 4 (start_index, end_index): (173, 193)
 top 5 (start_index, end_index): (173, 179)


[(0, 0, 18, 0.3386707305908203),
 (0, 0, 0, 0.22401131689548492),
 (0, 13, 18, 0.14949698746204376),
 (0, 17, 18, 0.013705562800168991),
 (0, 0, 19, 0.011742623522877693),
 (1, 173, 184, 0.9714868664741516),
 (1, 173, 185, 0.015565156936645508),
 (1, 166, 184, 0.007162699941545725),
 (1, 173, 193, 0.0008136184187605977),
 (1, 173, 179, 0.0007655616500414908)]

In [51]:
candidates.sort(key=lambda x: x[-1], reverse=True)
candidates_top5 = candidates[:5]
candidates_top5

[(1, 173, 184, 0.9714868664741516),
 (0, 0, 18, 0.3386707305908203),
 (0, 0, 0, 0.22401131689548492),
 (0, 13, 18, 0.14949698746204376),
 (1, 173, 185, 0.015565156936645508)]

In [55]:
for candidate in candidates_top5:
    offset_idx, start_token, end_token, score = candidate
    offset = offsets[offset_idx]
    start_char, _ = offset[start_token]
    _, end_char = offset[end_token]
    answer = long_context[start_char:end_char]
    result = {"answer": answer, "start": start_char, "end": end_char, "score": score}
    print(result)

{'answer': 'Jax, PyTorch and TensorFlow', 'start': 1892, 'end': 1919, 'score': 0.9714868664741516}
{'answer': '\n🤗 Transformers: State of the Art NLP', 'start': 0, 'end': 37, 'score': 0.3386707305908203}
{'answer': '', 'start': 0, 'end': 0, 'score': 0.22401131689548492}
{'answer': 'State of the Art NLP', 'start': 17, 'end': 37, 'score': 0.14949698746204376}
{'answer': 'Jax, PyTorch and TensorFlow —', 'start': 1892, 'end': 1921, 'score': 0.015565156936645508}
